In [1]:
from neo4j import GraphDatabase
import pandas as pd

def read_csv(csv_file):
    data = pd.read_csv(csv_file)
    return data.to_dict(orient='records')

def import_data_to_neo4j(uri, username, password, universities_csv, courses_csv, students_csv, searches_csv):
    driver = GraphDatabase.driver(uri, auth=(username, password))

    universities_data = read_csv(universities_csv)
    courses_data = read_csv(courses_csv)
    students_data = read_csv(students_csv)
    searches_data = read_csv(searches_csv)

    university_query = """
    UNWIND $data as row
    MERGE (u:University {id: row.id})
    SET u.name = row.name,
        u.country = row.country
    """

    course_query = """
    UNWIND $data as row
    MATCH (u:University {id: row.uni})
    MERGE (c:Course {id: row.id})
    SET c.name = row.name,
        c.year = row.year,
        c.semester = row.semester
    MERGE (c)-[:BELONGS_TO]->(u)
    """

    student_query = """
    UNWIND $data as row
    MERGE (s:Student {id: row.id})
    SET s.name = row.userid_DI,
        s.country = row.final_cc_cname_DI,
        s.education = row.LoE_DI,
        s.gender = row.gender,
        s.age = row.age
    """


    search_query = """
    UNWIND $data as row
    MATCH (s:Student {id: row.user})
    MERGE (search:Search {id: row.id})
    SET search.date = row.date,
        search.words = row.words
    MERGE (s)-[:SEARCHED]->(search)
    """

    def execute_query(tx, query, data):
        tx.run(query, data=data)

    with driver.session() as session:
        session.write_transaction(execute_query, university_query, universities_data)
        session.write_transaction(execute_query, course_query, courses_data)
        session.write_transaction(execute_query, student_query, students_data)
        session.write_transaction(execute_query, search_query, searches_data)

    driver.close()


uri = "bolt://localhost:7687"
username = "neo4j"  
password = "12345678"  

universities_csv = "univ.csv"
courses_csv = "course.csv"
students_csv = "student.csv"
searches_csv = "search.csv"


import_data_to_neo4j(uri, username, password, universities_csv, courses_csv, students_csv, searches_csv)


C:\Users\Ghada\AppData\Local\Temp\ipykernel_29780\108645062.py:65: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(execute_query, university_query, universities_data)
C:\Users\Ghada\AppData\Local\Temp\ipykernel_29780\108645062.py:66: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(execute_query, course_query, courses_data)
C:\Users\Ghada\AppData\Local\Temp\ipykernel_29780\108645062.py:67: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(execute_query, student_query, students_data)
C:\Users\Ghada\AppData\Local\Temp\ipykernel_29780\108645062.py:68: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(execute_query, search_query, searches_data)
